In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import os
import datetime as dt
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
#Define the function to access fields within a layer --> provides a pandas dataframe of the data table
def access_lyr(content,out_fld):
    base = content.layers
    lyr = base[0]
    topic_data = lyr.query(where="CommunityName = '" + str(community.replace("'","''")) + "'" ,out_fields=out_fld)
    return topic_data.df

today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Communities\Update Main Photo"
log_file = os.path.join(log_path, "Update Main Photo_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')

#Find storymaps to edit
find_storymaps = gis.content.search("title:* Community Storymap*", item_type="Web Mapping Application", max_items=500, outside_org=False)

In [ ]:
for f in find_storymaps:
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
            print(f.title, f.id)
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            print(community)
            #Access the URL for the main photo from the Photo Link feature class
            try:
                photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
                find_photo = access_lyr(photo_base,'PhotoLink')
                main_pic = find_photo.PhotoLink[0]
            except:
                print('______Issues with connecting to data.______')

            #Define the main panel to change the main image content
            panel_toc = sections[0]
            #Change the photo by searching for a link from the photo links feature class. If not there, use the default Denali
            #photo
            print("Updating main picture...")
            panel_toc["title"] = '<span style="font-size:40px">' + str(community) + ', Alaska</span>'
            if str(main_pic) == 'None':
                panel_toc["media"]["image"]["url"] = "https://maps.commerce.alaska.gov/portal/sharing/rest/content/items/0de9a1aac2804454978e697354ec321f/data"
            else:
                panel_toc["media"]["image"]["url"] = main_pic
            csmap.save()
            print("Photo updated.")
    except:
        print("-------ISSUE WITH" + str(f.title) + "--------")
        out.write('Unable to update photo for ' + str(f.title))
        out.write('\n\n')
        continue